In [1]:
import json
from pathlib import Path

import chromadb

In [ ]:
storage_path = "vector_storage"  # directory, where ChromaDB will store collections
persistent_client = chromadb.PersistentClient(path=storage_path)
legal_practices_coll = persistent_client.create_collection(
    name="legal_practices", get_or_create=True, metadata={"hnsw:space": "cosine"}
)

In [ ]:
legal_practices_data_path = Path("data/summaries_w_emb.json")  # dict with docs uid, summary and embeddings 
with open(legal_practices_data_path, "r") as f:
    legal_practices_data = json.load(f)

In [ ]:
legal_practices_meta_path = Path("data/doc2meta.json")  # dict with docs uid and metadata (codex and theme)
with open(legal_practices_meta_path, "r") as f:
    legal_practices_meta = json.load(f)

In [ ]:
def squeeze_meta(meta: dict) -> dict:
    """
    Convert metadatas lists to strings
    """
    meta["codex"] = "".join(meta["codex"])
    meta["theme"] = "\n".join(meta["theme"])
    return meta

In [21]:
legal_practices_coll.add(
    ids=[k for k in legal_practices_data.keys()],
    embeddings=[val["emb"] for val in legal_practices_data.values()],
    metadatas=[squeeze_meta(legal_practices_meta[k]) for k in legal_practices_data.keys()],
    documents=[val["text"] for val in legal_practices_data.values()],
)